In [2]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import random
import math
import PIL
from PIL import Image
import matplotlib.image as mpimg

The blocks here contain miscellaneous methods for image processing.

In [3]:
#im going to be honest i dont remember why i made this and also if i even used it
#regardless, here it lies, and here it shall lie indeffinitely
#unless i decide to remove it
def conv(img, kernel):
    kH,kW = kernel.shape
    (imH,imW) = img.shape
    new_img = np.zeros(img.shape)
    pad = int((kH-1)/2)

    for y in range(imH-kH):
        for x in range(imW-kW):
            window = img[y:y+kH,x:x+kW]
            new_img[y+pad,x+pad] = (kernel * window).sum()

    return new_img

In [7]:
#replace the path it a path to a folder containing images
#it'll put the images in an array and you can access them later.
def get_image():
    DATADIR = "your path here"


    selected = False
    while(selected == False):
        rand = random.randrange(0,len(os.listdir(DATADIR)))
        img_name = (os.listdir(DATADIR))[rand]
        if img_name.endswith("JPG" or "jpg" or "png" or "PNG"):
            # read an image
            img = Image.open(os.path.join(DATADIR, (os.listdir(DATADIR))[rand])).convert('L')
            Oimg = mpimg.imread(os.path.join(DATADIR, (os.listdir(DATADIR))[rand]))
            img_arr = np.asarray(img)
            selected = True

    return img_arr, Oimg

In [ ]:
import numpy as np
import math
import cv2
from tqdm import tqdm


#this is the sobel filter, it also applies non-maximal-suppression (nms)
#for those that dont know, sobel is a contrast based edge detection algorithm
#and nms is a processing step that thins the underlying structure found by sobel
#from this point forward all processing is done on images in the form of ndarrays
def sobel_filter(img):
    sobelx = np.array([[-1, 0, 1],
                       [-2, 0, 2],
                       [-1, 0, 1]])

    sobely = np.array([[-1, -2, -1],
                       [0, 0, 0],
                       [1, 2, 1]])

    kH, kW = sobelx.shape
    (imH, imW) = img.shape
    new_img = np.zeros(img.shape)
    theta = np.zeros(img.shape)
    hsv_img = np.zeros((img.shape[0], img.shape[1], 3))
    nms_img = np.zeros(img.shape)
    pad = int((kH - 1) / 2)

    # Add progress bar for the first loop
    for y in tqdm(range(imH - kH), desc='Applying Sobel filter'):
        for x in range(imW - kW):
            window = img[y:y + kH, x:x + kW]
            new_img[y + pad, x + pad] = (((sobelx * window).sum()) ** 2 + ((sobely * window).sum()) ** 2) ** 0.5
            if new_img[y + pad, x + pad] > 0:
                if (sobelx * window).sum() == 0:
                    theta[y + pad, x + pad] = 90
                else:
                    theta[y + pad, x + pad] = (math.degrees(math.atan((sobely * window).sum() / (sobelx * window).sum()))) % 180



    # Add progress bar for the second loop
    for i in tqdm(range(imH - 1), desc='Applying Non-Maximum Suppression'):
        for j in range(imW - 1):
            if theta[i, j] != 0:
                quad = theta[i, j] // 22.5
                neighborA = 255
                neighborB = 255

                if quad in [0, 8]:
                    neighborA = new_img[i, j - 1]  # Left
                    neighborB = new_img[i, j + 1]  # Right
                elif quad in [1, 2]:
                    neighborA = new_img[i - 1, j - 1]  # LowerLeft
                    neighborB = new_img[i + 1, j + 1]  # UpperRight
                elif quad in [5, 6]:
                    neighborA = new_img[i - 1, j + 1]  # LowerRight
                    neighborB = new_img[i + 1, j - 1]  # UpperLeft
                elif quad in [3, 4]:
                    neighborA = new_img[i - 1, j]  # Down
                    neighborB = new_img[i + 1, j]  # Up
                if max([neighborA, new_img[i, j], neighborB]) == new_img[i, j] and new_img[i, j] >= 255 * 0.4:
                    nms_img[i, j] = new_img[i, j]
                    hsv_img[i, j, :] = [theta[i, j], 255, 255]

    return new_img.astype(np.uint8), hsv_img.astype(np.uint8), nms_img.astype(np.uint8)


In [8]:
img, Oimg = get_image()
img = cv2.GaussianBlur(img,(3,3),0)

In [ ]:
plt.figure()
plt.title("Original Image")
plt.imshow(Oimg)
plt.figure()
plt.title("Original Image (Grayscale)")
plt.imshow(img, 'gray')

In [ ]:
sobel_img,hsv_img,nms_img = sobel_filter(img)

plt.figure()
plt.title("Original Image (grayscale)")
plt.imshow(img,cmap='gray')
plt.figure()
plt.title("After applying Sobel Filter")
plt.imshow(sobel_img,cmap='gray')
plt.figure()
plt.title("After applying NMS and Double Threshold")
plt.imshow(nms_img,cmap='gray')
plt.figure()
plt.title("Color representation of gradient direction")
plt.imshow(cv2.cvtColor(hsv_img,cv2.COLOR_HSV2RGB))

In [ ]:
count = 0
blur = cv2.GaussianBlur(img,(3,3),0.5)
plt.figure(figsize=(10,10))
while(True):
    blur = cv2.GaussianBlur(blur,(3,3),0)
    new_blur = cv2.GaussianBlur(blur,(3,3),0)
    count+=1
    plt.subplot(2,3,count)
    plt.imshow((blur-new_blur),cmap="gray")
    if(count == 6):
        break
    blur = new_blur
plt.tight_layout()
plt.show()

Experiment starts here

boost sobel map because nms output is too weak

In [ ]:
plt.figure()
plt.title("After applying Sobel Filter")
plt.imshow(sobel_img,cmap='gray')


In [ ]:
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output

def threshold_map(image, threshold):
  """
    Boosts the highlights and darkens the shadows of an image.

    Args:
      image: A 2D NumPy array representing the image.
      threshold: The threshold value.

    Returns:
      A 2D NumPy array representing the boosted image.
    """

  # Create a copy of the image.
  copy_image = image.copy()

  # Create a progress bar
  progress_bar = tqdm(total=copy_image.size, desc="Processing")

  # Boost the highlights and darken the shadows.
  for i in range(len(copy_image)):
    for j in range(len(copy_image[0])):
      if copy_image[i][j] >= threshold:
        copy_image[i][j] = 100
      else:
        copy_image[i][j] = 0

      # Update the progress bar
      progress_bar.update(1)

  # Close the progress bar
  progress_bar.close()

  # Return the boosted image.
  return copy_image



In [ ]:
mask=threshold_map(sobel_img, 18)

In [ ]:
plt.figure()
plt.title("Sobel Edge Map")
plt.imshow(sobel_img,cmap='gray')
plt.figure()
plt.title("Threshold Mask")
plt.imshow(mask,cmap='gray')

Apply canny

In [17]:
import numpy as np
from scipy import ndimage
from skimage.feature import canny
#i didnt write the code for canny because it was easily available
#the other steps may also now be abvailable
#but ive already written them and i cannot be bothered to remove them
def apply_canny_edge_detection(edge_map):
    # Convert the edge map to a uint8 grayscale image
    image = np.array(edge_map, dtype=np.uint8)

    # Apply Canny edge detection
    edges = canny(image)

    # Convert the boolean array of edges to integer values (0 and 255)
    #edges = edges.astype(np.int64) * 255

    # Use morphological operations to enhance the edges (optional)
    #edges = ndimage.binary_dilation(edges, iterations=2).astype(np.uint8) * 255

    return edges


In [18]:
cannyoutput=apply_canny_edge_detection(mask)

In [19]:
cannyS=apply_canny_edge_detection(sobel_img)

In [ ]:
plt.figure()
plt.title("Sobel Edge Map")
plt.imshow(sobel_img,cmap='gray')
plt.figure()
plt.title("Threshold Mask")
plt.imshow(mask,cmap='gray')
plt.figure()
plt.title("Canny on Threshold Mask")
plt.imshow(cannyoutput,cmap='gray')
plt.figure()
plt.title("Canny on Sobel Map")
plt.imshow(cannyS,cmap='gray')

close any open loops 

In [22]:
# Perform morphological closing
# Morphological closing closes any open loops in the image
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 20))  # Adjust kernel size as needed
closed_image = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

In [23]:
# Perform image filling
# Filling fills the aforementioned loops
filled_image = closed_image.copy()
contours, _ = cv2.findContours(closed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
    cv2.drawContours(filled_image, [contour], 0, 255, -1)

In [ ]:
plt.figure()
plt.title("Threshold Mask")
plt.imshow(mask,cmap='gray')
plt.figure()
plt.title("Closed mask")
plt.imshow(closed_image,cmap='gray')
plt.figure()
plt.title("Filled mask")
plt.imshow(filled_image,cmap='gray')

In [ ]:
import numpy as np
from scipy.ndimage import label

# This block finds objects in the image as clusters, and as a requirement
# youhave to have a compy of you image in a binary representation (subjects/objects on white background, or vice versa).
# you can use this to create a mask to modify selective parts of the image
# the way this code was written, it gets rid of all clusters under a certain size

# Convert image to binary
threshold_percentage = 0.01  # Remove clusters smaller than 20% of the image
threshold = int(threshold_percentage * np.prod(filled_image.shape))
binary_image = filled_image > 0

# Label connected components
labeled_image, num_labels = label(binary_image)

# Calculate sizes of each label
label_sizes = np.bincount(labeled_image.ravel())
small_clusters = np.where(label_sizes < threshold)[0]

# Create mask to remove small clusters
mask = np.isin(labeled_image, small_clusters)

# Apply mask to the original image
result = filled_image.copy()
result[mask] = 0

plt.figure()
plt.title("Denoised")
plt.imshow(result,cmap='gray')


In [ ]:
# Create a copy of the original image
imgcpy = Oimg.copy()

# this demonstrates how you can use this pipeline to remove backgrounds from product images

# Iterate through the filled image and update the copy
for i in tqdm(range(len(result)), desc='Processing'):
    for j in range(len(result[0])):
        if result[i][j] == 0:
            imgcpy[i][j] = 255



In [ ]:
plt.figure()
plt.title("Original Image")
plt.imshow(Oimg)
plt.figure()
plt.title("Trimmed Image")
plt.imshow(imgcpy,cmap='gray')

In [46]:
# This is a fun one. this finds the min/max x,y values for nonwhite pixels, meaning
# you can use this to crop out empty spaces

def find_min_max_nonwhite(imgcpy):
  """
  Finds the min/max values for x/y for the entries that are not white in a NumPy ndarray (imgcpy).

  Args:
    imgcpy: A NumPy ndarray that represents an image.

  Returns:
    A tuple of (min_x, max_x, min_y, max_y).
  """

  # Create a boolean mask that will identify the non-white entries in the image.
  mask = imgcpy != 255

  # Find the minimum and maximum values of the x and y coordinates of the non-white entries.
  min_x = np.min(mask.nonzero()[0])
  max_x = np.max(mask.nonzero()[0])
  min_y = np.min(mask.nonzero()[1])
  max_y = np.max(mask.nonzero()[1])

  return min_x, max_x, min_y, max_y

In [48]:
min_x, max_x, min_y, max_y = find_min_max_nonwhite(imgcpy)
print(min_x, max_x, min_y, max_y)


1046 2182 169 3712


In [ ]:

# Draw lines to mark new boundaries on the input image
img_marked = imgcpy.copy()
cv2.line(img_marked, (min_y, min_x), (max_y, min_x), (255, 0, 0), 2)
cv2.line(img_marked, (max_y, min_x), (max_y, max_x), (255, 0, 0), 2)
cv2.line(img_marked, (max_y, max_x), (min_y, max_x), (255, 0, 0), 2)
cv2.line(img_marked, (min_y, max_x), (min_y, min_x), (255, 0, 0), 2)

plt.figure()
plt.title("Croped Image")
plt.imshow(img_marked,cmap='gray')

In [ ]:
# Crop the image based on the highest/lowest x/y values
cropped_img = imgcpy[min_x:max_x+1, min_y:max_y+1, :]

# Calculate the new dimensions
new_height, new_width, _ = cropped_img.shape

# Resize the image to the new dimensions
resized_img = cv2.resize(cropped_img, (new_width, new_height))

plt.figure()
plt.title("Resized Image")
plt.imshow(resized_img,cmap='gray')

In [33]:
#  the purpose of this was to fix the colors of parts that are transparent
#(the glasses in the example(this is not a very good example,
#but sometimes with closed loop frames, the glass section retains the colors fo the backgroun))

def lighten_image(image, L):
  """
  Lightens all the pixels in an image with a value higher than L.

  Args:
    image: The input image.
    L: The threshold value.

  Returns:
    The output image after lightening.
  """
  image_array=image.copy()

  # Calculate factor
  # Lighten the high pixels.
  for i in image_array:
      for j in image_array[i]:
          if image_array[i][j]>L:
              image_array[i][j]=image_array[i][j]+((255-image_array[i][j])*0.8)

  # Convert the NumPy array back to an image.
  output_image = Image.fromarray(image_array)

  return output_image


In [ ]:
limgcpy=lighten_image(imgcpy, 150)


# i just noticed this doesnt work
# i probably made a change at some point ant promptly gave up
# maybe ill fix it, maybe i wont
# guess well find out eventually

In [ ]:
plt.figure()
plt.title("Trimmed Image")
plt.imshow(imgcpy,cmap='gray')
plt.figure()
plt.title("Lightened Image")
plt.imshow(limgcpy)

In [25]:
# Create a PIL Image from the NumPy array
image_pil = Image.fromarray(imgcpy)

# Save the image
image_pil.save("output_image.jpg")  # Adjust the filename and format as needed